In [ ]:
# !pip install sqlite3

In [ ]:
import pandas as pd
from tqdm import tqdm

# И задаем запрос

In [129]:
import sqlite3

def get_sentences_with_lemma_only(db_file, lemma):
    """Выбирает предложения из таблицы SENT,
    которые содержат слова с заданной леммой.
    """

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Запрос с несколькими JOIN
    cursor.execute("""
        SELECT DISTINCT WORD.sent_id, WORD.word_position
        FROM WORD
        JOIN SENT ON SENT.sent_id = WORD.sent_id
        WHERE WORD.lemma = ?
    """, (lemma,))

    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    return result

In [130]:
def get_sentences_with_wordform_only(db_file, wordform):
    """Выбирает предложения из таблицы SENT,
    которые содержат слова с заданной словоформой.
    """

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    # Запрос с несколькими JOIN
    cursor.execute("""
        SELECT DISTINCT WORD.sent_id, WORD.word_position
        FROM WORD
        JOIN SENT ON SENT.sent_id = WORD.sent_id
        WHERE WORD.word = ?
    """, (wordform,))

    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    return result

In [131]:
def get_sentences_with_lemma_tag(db_file, lemma, tag):
    """Выбирает предложения из таблицы SENT,
    которые содержат слова с заданной леммой и тегом.
    """

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Запрос с несколькими JOIN
    cursor.execute("""
        SELECT DISTINCT WORD.sent_id, WORD.word_position
        FROM WORD
        JOIN SENT ON SENT.sent_id = WORD.sent_id
        WHERE WORD.lemma = ? and WORD.tag = ?
    """, (lemma, tag))

    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    return result

In [132]:
def get_sentences_with_wordform_tag(db_file, wordform, tag):
    """Выбирает предложения из таблицы SENT,
    которые содержат слова с заданной словоформой и тегом.
    """

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    # Запрос с несколькими JOIN
    cursor.execute("""
        SELECT DISTINCT WORD.sent_id, WORD.word_position
        FROM WORD
        JOIN SENT ON SENT.sent_id = WORD.sent_id
        WHERE WORD.word = ? and WORD.tag = ?
    """, (wordform, tag))

    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    return result

In [133]:
def get_sentences_with_tag(db_file, tag):
    """Выбирает предложения из таблицы SENT,
    которые содержат слова с заданным тегом.
    """

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    # Запрос с несколькими JOIN
    cursor.execute("""
        SELECT DISTINCT WORD.sent_id, WORD.word_position
        FROM WORD
        JOIN SENT ON SENT.sent_id = WORD.sent_id
        WHERE WORD.tag = ?
    """, (tag, ))

    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    return result

In [35]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 56.8 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=e4fc95ee89aa5420c43922f1af21b1d2fa40f725346da83a0567c678da6d6f08
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [36]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

In [112]:
all_tags = ['A', 'ADV', 'ADVPRO', 'ANUM', 'APRO',
            'COM', 'CONJ', 'INTJ', 'NUM', 'PART',
            'PR', 'S', 'SPRO', 'V']

In [134]:
def search(query, db_file):
    """Выбирает предложения из таблицы SENT,
    которые содержат поисковый запрос.
    """

    # делим запрос на части
    query_parts = query.split()
    # счетчик по предложениям
    wp = None
    # для каждой части запроса
    for query_part in query_parts:
        # делим запрос по знаку + (если вдруг есть тег)
        part = query_part.split('+')
        # если есть тег
        if len(part) == 2:
            # отдельно записываем слово и тег
            word, tag = part[0], part[1]
            # если мы ищем словоформу
            if word[0] and word[-1] == '"':
                # находим все предложения со словоформой
                # (предложение, номер в предложении)
                sentences = get_sentences_with_wordform_tag(db_file, word, tag)
                # если счетчик не задан (первый элемент запроса)
                if wp == None:
                    # делаем словарь
                    # предложение: номер в предложении
                    wp = {s[0]: [s[1]] for s in sentences}
                # если уже задан (то есть не первый элемент)
                else:
                    for s in sentences:
                        if s[0] in wp:
                            if wp[s[0]][-1] == s[1] - 1:
                                wp[s[0]].append(s[1])
                            else:
                                del wp[s[0]]
            # если ищем лемму
            else:
                # собственно лемма
                word = morph.parse(word)[0].normal_form
                # дальше то же самое
                sentences = get_sentences_with_lemma_tag(db_file, word, tag)
                if wp == None:
                    wp = {s[0]: [s[1]] for s in sentences}
                    # sents.append(sentences)
                else:
                    for s in sentences:
                        if s[0] in wp:
                            if wp[s[0]][-1] == s[1] - 1:
                                wp[s[0]].append(s[1])
                            else:
                                del wp[s[0]]
        # если тега нет
        else:
            # то же самое, что раньше
            word = part[0]
            if word[0] and word[-1] == '"':
                sentences = get_sentences_with_wordform_only(db_file, word)
                if wp == None:
                    wp = {s[0]: [s[1]] for s in sentences}
                    # sents.append(sentences)
                else:
                    for s in sentences:
                        if s[0] in wp:
                            if wp[s[0]][-1] == s[1] - 1:
                                wp[s[0]].append(s[1])
                            else:
                                del wp[s[0]]
            else:
                if word not in all_tags:
                    word = morph.parse(word)[0].normal_form
                    sentences = get_sentences_with_lemma_only(db_file, word)
                else:
                    sentences = get_sentences_with_tag(db_file, word)
                if wp == None:
                    wp = {s[0]: [s[1]] for s in sentences}
                    # sents.append(sentences)
                else:
                    for s in sentences:
                        if s[0] in wp:
                            if wp[s[0]][-1] == s[1] - 1:
                                wp[s[0]].append(s[1])
                            else:
                                del wp[s[0]]
    sents = [elem for elem in wp if len(wp[elem]) == len(query_parts)]
    # sents = list(wp.keys())
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    # Запрос с несколькими JOIN
    cursor.execute("""
        SELECT DISTINCT sentence_text
        FROM SENT
        WHERE sent_id IN ({seq})""".format(
    seq=','.join(['?']*len(sents))), sents)

    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()
    return result

In [135]:
search('A рамка', 'sem_notes_corp.db')

[('Модальная рамка.',),
 ('Это не пресуппозиция и не ассерция: сочетание отрицания с, например, словами, обеспечивающими модальную рамку, в принципе аномально (см.',),
 ('Заметим, что *Он выпил не всего два бокала вина нельзя, слова с модальной рамкой с отрицанием не сочетаются.',),
 ('PPI не сочетаются с отрицанием, потому что многие PPI – слова, состоящие из одной модальной рамки.',)]